In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
params = {"grupo":"14"}

In [4]:
r = requests.get(base_url, params=params)

In [5]:
page = BeautifulSoup(r.text, 'html.parser')

In [6]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [7]:
__EVENTVALIDATION

'/wEdAAxXJ2MkySX1N58GqVoE0V564mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc0g5Yr4HVWZGcKVTKdqN6QTVRU9dw=='

In [8]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [9]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUCMTQeCGlkZkdydXBvAg4eCGJ1c2NhUG9yZBYCZg9kFgICAw9kFgICAQ8WAh4GYWN0aW9uBR8vZGFkb3NhYmVydG9zL2NvbnN1bHRhP2dydXBvPTE0Fg4CAg8QDxYGHg1EYXRhVGV4dEZpZWxkBQpOb21lUGFnaW5hHg5EYXRhVmFsdWVGaWVsZAUJVXJsUGFnaW5hHgtfIURhdGFCb3VuZGcWBB4Jb25rZXlkb3duBUNpZihJc0VudGVyS2V5KGV2ZW50KSl7UmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpO3JldHVybiBmYWxzZTt9HgdvbmNsaWNrBR9SZWRpcmVjaW9uYXJBY2Vzc29SYXBpZG8odGhpcyk7EBUGDkFjZXNzbyBSw6FwaWRvFlNlY3JldGFyaWFzIGUgw5NyZ8Ojb3MRU2VydmnDp29zIE9uLUxpbmUYU2VydmnDp29zIHBhcmEgYSBFbXByZXNhGVNlcnZpw6dvcyBwYXJhIG8gQ2lkYWTDo28TU2l0ZXMgZGEgUHJlZmVpdHVyYRUGAA0vc2VjcmV0YXJpYXMvGS9saXN0YS1zZXJ2aWNvcy1vbi1saW5lLzIRL3NlcnZpY29zL2VtcHJlc2ERL3NlcnZpY29zL2NpZGFkYW8cL2xpc3RhLXNpdGVzLWRhLXByZWZlaXR1cmEvNRQrAwZnZ2dnZ2dkZAIEDw9kFgIfBwVJcmV0dXJuIFZhbGlkYXJQZXNxdWlzYSgndHh0UGVzcXVpc2FUb3BvJywnY3RsMDAkbG5rUGVzcXVpc2FUb3BvJywgZXZlbnQpO2QCBg8PFgIeBFRleHQFDlPDoWIsIDEzLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQU

In [10]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

c3d7c7962b0f35bdb3a4f1561341db99


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [18]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 4/4 [00:00<00:00, 9592.46it/s]


In [19]:
databases

[{'database_id': 'c3d7c7962b0f35bdb3a4f1561341db99',
  'database_title': 'legisladoc - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/Legisladoc/2018-10-12_legisladoc_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 12)}]

In [22]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/Legisladoc/2018-10-12_legisladoc_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,2,3,4,15,16,17,18,19,20,21,22,23,24,26,28,29,30,34,35,38,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
test.fillna('').head()

,ATO_IDF,ATO_NUMERO_IDENTIFICACAO,ATO_NUMERO_OFICIAL,ATO_DIARIO_OFICIAL,ATO_VERSAO,ATO_DATA_CRIACAO,ATO_DATA_ASSINATURA,ATO_DATA_PUBLICACAO,ATO_DATA_REPUBLICACAO,TIPO_ATO,...,DIARIO_OFICIAL_SUPLEMENTO_NUMERO,DIARIO_OFICIAL_SUPLEMENTO_ANO,DIARIO_OFICIAL_SUPLEMENTO_DATA_PUBLICACAO,DIARIO_OFICIAL_SUPLEMENTO_STATUS,DIARIO_OFICIAL_SUPLEMENTO_TEXTO,TIPO_ATO_ZERA_NUMERACAO,TIPO_ATO_TIPO_NUMERACAO,TIPO_ATO_ORDEM_APRESENTACAO,TIPO_ATO_GRUPO,TIPO_ATO_SITUACAO
0,--------------------,------------------------,--------------------,------------------,----------,----------------------------------------------...,----------------------------------------------...,------------------------------,----------------------------------------------...,----------------------------------------------...,...,--------------------------------,------------------------------,-----------------------------------------,--------------------------------,----------------------------------------------...,-----------------------,-----------------------,---------------------------,------------------------------,-----------------
1,148116,DOC148116,1922,8,1,07/12/2012 15:39:33,20/12/2012 15:36:50,20/12/2012,,DECRETO,...,,,,Fechado,,SIM,Numeração Manual,53,,Ativo
2,148118,DOC148118,730,7,1,10/12/2012 09:14:25,19/12/2012 13:11:58,19/12/2012,,PORTARIA,...,,,,Fechado,,SIM,Numeração Automática,61,,Ativo
3,148120,DOC148120,1926,2,1,10/12/2012 09:44:16,12/12/2012 16:24:20,13/12/2012,,DECRETO,...,,,,Fechado,,SIM,Numeração Manual,53,,Ativo
4,148121,DOC148121,19,1,1,10/12/2012 09:52:43,10/12/2012 17:21:51,11/12/2012,,AVISO DE LICITAÇÃO,...,,,,Fechado,,SIM,Numeração Manual,4,Licitações,Ativo
